In [11]:
import json
import pandas as pd

# Load the JSON file
with open('MMHS150K_GT.json') as f:
    data = json.load(f)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data).T  # Transpose the data to get proper structure
df.head()


,img_url,labels,tweet_url,tweet_text,labels_str
1114679353714016256,http://pbs.twimg.com/tweet_video_thumb/D3gi9MH...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,@FriskDontMiss Nigga https://t.co/cAsaLWEpue,"[Religion, Racist, Homophobe]"
1063020048816660480,http://pbs.twimg.com/ext_tw_video_thumb/106301...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,My horses are retarded https://t.co/HYhqc6d5WN,"[OtherHate, OtherHate, OtherHate]"
1108927368075374593,http://pbs.twimg.com/media/D2OzhzHUwAADQjd.jpg,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[NotHate, NotHate, NotHate]"
1114558534635618305,http://pbs.twimg.com/ext_tw_video_thumb/111401...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[Racist, NotHate, NotHate]"
1035252480215592966,http://pbs.twimg.com/media/Dl30pGIU8AAVGxO.jpg,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,“EVERYbody calling you Nigger now!” https://t....,"[Racist, NotHate, Racist]"


In [12]:
sexist_tweets = df[df['labels_str'].apply(lambda x: 'Sexist' in x)]


In [13]:
racist_tweets = df[df['labels_str'].apply(lambda x: 'Racist' in x)]

In [14]:
homophobic_tweets = df[df['labels_str'].apply(lambda x: 'Homophobe' in x)]

In [15]:
disability_keywords = ['retard', 'cripple', 'spaz', 'invalid', 'gimp', 'crazy', 'freak']
disability_tweets = df[df['tweet_text'].str.contains('|'.join(disability_keywords), case=False, na=False)]


In [16]:
subset_df = df.sample(frac=0.01)

In [8]:
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize tweet text
tokens = tokenizer(subset_df['tweet_text'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Extract text embeddings
outputs = model(**tokens)
text_embeddings = outputs.last_hidden_state


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ae5efkbp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ae5efkbp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=adf8b54166d695e7954977fe8cfd0295bf3b85320bb2bb4f2c00311e7d0d950f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5fqfsdyw/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [11]:
# import torch
# import clip
# from PIL import Image

# # Load CLIP model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)

# # Load and preprocess image
# image = Image.open('/content/1023940590382268417.jpg')
# image_input = preprocess(image).unsqueeze(0).to(device)

# # Extract visual features
# with torch.no_grad():
#     image_features = model.encode_image(image_input)


100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 72.2MiB/s]


In [32]:
import torch
import clip

# Assuming 'text_embeddings' and 'image_features' are the embeddings for a tweet

# Step 1: Define saliency points
gender_saliency_points = [
    "bitch", "slut", "feminazi", "stay in the kitchen", "man up", "gold digger",
    "women are property", "catfight", "misandrist", "alpha male"
]
disability_saliency_points = [
    "retard", "retarded", "cripple", "crippled", "burden on society",
    "worthless", "crazy", "spaz", "spastic", "freak", "vegetable",
    "invalid", "better off dead", "gimp"
]
race_saliency_points = [
    "nigger", "monkey", "go back to your country", "illegal", "terrorist",
    "chink", "spic", "thug", "white supremacy", "genetic inferior"
]
lgbtqia_saliency_points = [
    "faggot", "dyke", "tranny", "that's so gay", "it’s just a phase",
    "sodomite", "no homo", "groomer", "burn in hell", "agenda"
]

# Step 2: Encode saliency points using CLIP's text encoder
def encode_saliency_points(points, model, device):
    text_tokens = clip.tokenize(points).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
    return text_features

# Encode saliency points for each dimension
gender_features = encode_saliency_points(gender_saliency_points, clip_model, device)
disability_features = encode_saliency_points(disability_saliency_points, clip_model, device)
race_features = encode_saliency_points(race_saliency_points, clip_model, device)
lgbtqia_features = encode_saliency_points(lgbtqia_saliency_points, clip_model, device)

In [33]:
import torch
from transformers import BertTokenizer, BertModel
import clip
from PIL import Image
import torch.nn as nn

# Load pre-trained BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize tweet text (replace with your actual text data)
tweet_texts = subset_df['tweet_text'].tolist()  # Assuming subset_df['tweet_text'] contains your text data
tokens = bert_tokenizer(tweet_texts, padding=True, truncation=True, return_tensors='pt')

# Extract text embeddings (CLS token representation for the whole sentence)
with torch.no_grad():
    outputs = bert_model(**tokens)
    text_embeddings = outputs.last_hidden_state[:, 0, :]  # Shape: (batch_size, 768)

# Create a linear layer to project BERT embeddings to 512 dimensions
projection_layer = nn.Linear(768, 512).to(device)

# Project the text embeddings to 512 dimensions
text_embeddings_projected = projection_layer(text_embeddings.to(device))  # Shape: (batch_size, 512)

# Load CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Load and preprocess image (replace with your actual image path)
image_path = '/content/1023942343202881536.jpg'
image = Image.open(image_path)
image_input = preprocess(image).unsqueeze(0).to(device)

# Extract visual features
with torch.no_grad():
    image_features = clip_model.encode_image(image_input)  # Shape: (1, 512)

# Compute cosine similarity between projected text embeddings and image features
similarity = torch.nn.functional.cosine_similarity(text_embeddings_projected, image_features, dim=-1)

# Print similarity
print(similarity)


tensor([ 0.0021, -0.0132, -0.0248,  ..., -0.0115,  0.0064,  0.0010],
       device='cuda:0', grad_fn=<SumBackward1>)


In [34]:
import torch.nn as nn

# Define a projection layer to map 768-dimensional BERT embeddings to 512 dimensions
projection_layer = nn.Linear(768, 512).to(device)

# Project the text embeddings to match the dimension of CLIP embeddings (512)
# Move text_embeddings to the same device as projection_layer before applying it
projected_text_embeddings = projection_layer(text_embeddings.to(device))

# Step 3: Compute cosine similarity for each dimension (with projected embeddings)
def compute_similarity(text_embeddings, image_features, saliency_features):
    # Initialize empty lists to store similarity scores
    text_similarities = []
    image_similarities = []

    # Iterate over saliency points and compute similarity for each
    for i in range(saliency_features.shape[0]):
        # Select the current saliency point features
        current_saliency_features = saliency_features[i].unsqueeze(0)  # Reshape to (1, 512)

        # Calculate cosine similarity for text and image with the current saliency point
        text_similarity = torch.nn.functional.cosine_similarity(text_embeddings, current_saliency_features, dim=-1)
        image_similarity = torch.nn.functional.cosine_similarity(image_features, current_saliency_features, dim=-1)

        # Append the similarity scores to the lists
        text_similarities.append(text_similarity)
        image_similarities.append(image_similarity)

    # Stack the similarity scores into tensors
    text_similarities = torch.stack(text_similarities)  # Shape: (num_saliency_points, num_text_embeddings)
    image_similarities = torch.stack(image_similarities) # Shape: (num_saliency_points, num_image_features which is 1)

    return text_similarities, image_similarities

# Now you can compute similarity as before, but with projected text embeddings
gender_text_sim, gender_image_sim = compute_similarity(projected_text_embeddings, image_features, gender_features)
disability_text_sim, disability_image_sim = compute_similarity(projected_text_embeddings, image_features, disability_features)
race_text_sim, race_image_sim = compute_similarity(projected_text_embeddings, image_features, race_features)
lgbtqia_text_sim, lgbtqia_image_sim = compute_similarity(projected_text_embeddings, image_features, lgbtqia_features)

In [35]:
# Step 4: Summarize detected saliency points for each dimension
def detect_saliency(similarity_scores, threshold=0.1):
    return (similarity_scores > threshold).nonzero(as_tuple=True)[0].tolist()  # Indices of high similarity

# Detect high-similarity points
gender_saliency_detected = detect_saliency(gender_text_sim)
disability_saliency_detected = detect_saliency(disability_text_sim)
race_saliency_detected = detect_saliency(race_text_sim)
lgbtqia_saliency_detected = detect_saliency(lgbtqia_text_sim)

# Output saliency points detected in each dimension
print("Gender Dimension - Saliency Points Detected:", gender_saliency_detected)
print("Disability Dimension - Saliency Points Detected:", disability_saliency_detected)
print("Race Dimension - Saliency Points Detected:", race_saliency_detected)
print("LGBTQIA+ Dimension - Saliency Points Detected:", lgbtqia_saliency_detected)

Gender Dimension - Saliency Points Detected: [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
Disability Dimension - Saliency Points Detected: []
Race Dimension - Saliency Points Detected: [4, 4, 4, 5, 9]
LGBTQIA+ Dimension - Saliency Points Detected: []
